In [2]:
import socket
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import pickle

# Configuración del servidor
server_ip = '10.1.1.1'
server_port = 12346

# Generar una clave aleatoria para cifrado AES
encryption_key = get_random_bytes(16)

# Inicializar el servidor
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((server_ip, server_port))
server_socket.listen(5)

print(f"Servidor en {server_ip}:{server_port}")

# Diccionario para mantener las sesiones de los clientes
client_sessions = {}

# Función para otorgar una nueva sesión a un cliente
def grant_new_session(client_socket):
    # Generar un ID de cliente único
    client_id = len(client_sessions) + 1

    # Enviar el ID del cliente al cliente
    client_socket.send(str(client_id).encode())

    # Serializar y enviar la clave de cifrado al cliente
    encrypted_key = pickle.dumps(encryption_key)
    client_socket.send(encrypted_key)

    # Agregar la sesión del cliente al diccionario
    client_sessions[client_id] = client_socket

# Función para reenviar mensajes a otros clientes
def forward_message(sender_id, message):
    for client_id, client_socket in client_sessions.items():
        if client_id != sender_id:
            try:
                client_socket.send(message)
            except Exception as e:
                print(f"Error al reenviar mensaje al cliente {client_id}: {e}")

# Esperar conexiones entrantes
while True:
    client_socket, client_address = server_socket.accept()
    print(f"Conexión entrante desde {client_address}")
    grant_new_session(client_socket)

    # Manejar mensajes entrantes de este cliente
    client_id = len(client_sessions)
    while True:
        try:
            encrypted_message = client_sessions[client_id].recv(1024)
            if not encrypted_message:
                break

            # Descifrar mensaje del cliente actual
            cipher = AES.new(encryption_key, AES.MODE_EAX, nonce=encrypted_message[:16])
            decrypted_message = cipher.decrypt(encrypted_message[16:])

            # Mostrar mensaje en el servidor
            print(f"Mensaje del cliente {client_id}: {decrypted_message.decode()}")

            # Reenviar mensaje a otros clientes
            forward_message(client_id, encrypted_message)

        except Exception as e:
            print(f"Error al recibir mensaje del cliente {client_id}: {e}")

# Cerrar conexiones al finalizar el servidor (este código nunca se ejecutará en este ejemplo)
for client_socket in client_sessions.values():
    client_socket.close()
server_socket.close()

Servidor en 10.1.1.1:12346
Conexión entrante desde ('10.1.1.3', 62363)
Mensaje del cliente 1: Belligooooooooooooooool Belligol Bellingham, E Ele
Conexión entrante desde ('10.1.1.2', 58979)
Mensaje del cliente 2: HALA MADRID Y NADA MAS!!!!.
Conexión entrante desde ('10.1.1.2', 58986)
Mensaje del cliente 3: HALA MADRID Y NADA MAS!!!!.
Error al reenviar mensaje al cliente 1: [WinError 10053] Se ha anulado una conexión establecida por el software en su equipo host
Conexión entrante desde ('10.1.1.2', 58987)
Mensaje del cliente 4: HALA MADRID Y NADA MAS!!!!.
Error al reenviar mensaje al cliente 1: [WinError 10053] Se ha anulado una conexión establecida por el software en su equipo host
Error al reenviar mensaje al cliente 2: [WinError 10053] Se ha anulado una conexión establecida por el software en su equipo host
Conexión entrante desde ('10.1.1.2', 58988)
Mensaje del cliente 5: HALA MADRID Y NADA MAS!!!!.
Error al reenviar mensaje al cliente 1: [WinError 10053] Se ha anulado una conexión e